In [14]:
from __future__ import print_function

import os.path

import pymongo

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [4]:
### google mail API

def gmail_authentication():
    SCOPES = ['https://mail.google.com/']
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return build('gmail', 'v1', credentials=creds)

In [19]:
service = gmail_authentication()
messages_id = service.users().messages()\
    .list(userId='me', maxResults=10).execute()

In [28]:
### edit form for mongodb
message_result = []
for message in messages_id['messages']:
    msg = service.users().messages().get(userId='me', id=message['id']).execute()
    tmp_dict = {
        'id' : msg.get('id'),
        'labelIds' : msg.get('labelIds'),
        'snippet' : msg.get('snippet')
    }
    headers = msg['payload']['headers']
    for item in headers:
        tmp_dict['headers.'+item['name']] = item['value']
    message_result.append(tmp_dict)

In [29]:
message_result

[{'id': '186a5e4bdbe6bede',
  'labelIds': ['UNREAD', 'CATEGORY_UPDATES', 'INBOX'],
  'snippet': 'BCcard BC카드 페이북쇼핑 페이북 TOP포인트 소멸 안내 안녕하세요. 이수민 고객님. 항상 비씨카드를 애용해 아끼고 사랑해 주시는 고객님께 감사의 말씀을 드립니다. 고객님의 소중한 TOP 포인트가 유효기간 만료로 인해 소멸 예정이오니, TOP포인트 유효 기간 내 사용하시기 바랍니다. 소멸예정 포인트 (추출 기준일자 : 2023.02.28) 소멸예정포인트',
  'headers.Delivered-To': 'sooh0601@gmail.com',
  'headers.Received': 'from ems.bccard.com (bcems5.bccard.com. [211.181.255.25])        by mx.google.com with SMTP id kz6-20020a170902f9c600b00189ad19dd26si1270297plb.326.2023.03.02.21.14.36        for <sooh0601@gmail.com>;        Thu, 02 Mar 2023 21:14:36 -0800 (PST)',
  'headers.X-Google-Smtp-Source': 'AK7set8eAzQin28vxmMF55r+oD/4RtFza8LtEn+4mhOpe2CcU6UIErEByOUUuezKLBGZ6Gboifpp',
  'headers.X-Received': 'by 2002:a17:903:485:b0:19e:2fc7:573 with SMTP id jj5-20020a170903048500b0019e2fc70573mr3612292plb.30.1677820476784;        Thu, 02 Mar 2023 21:14:36 -0800 (PST)',
  'headers.ARC-Seal': 'i=1; a=rsa-sha256; t=1677820476; cv=none;        d=goog

In [30]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017/?readPreference=primary&ssl=false&directConnection=true")
db = client['']